In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import mean_absolute_percentage_error
# from xgboost import XGBRegressor
import tensorflow as tf
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
# from lightgbm import LGBMRegressor
# from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.pipeline import make_pipeline
from sklearn.neighbors import KNeighborsRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF, ConstantKernel as C
from sklearn.linear_model import TheilSenRegressor
from sklearn.linear_model import RANSACRegressor, LinearRegression
from sklearn.ensemble import StackingRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
# from pytorch_tabnet.tab_model import TabNetRegressor

import torch
# import autosklearn.regression
# from tpot import TPOTRegressor
# import h2o
# from h2o.automl import H2OAutoML

In [ ]:
def TSR(data, target):
    """Theil-Sen Regression with MAPE evaluation."""

    # Split data into features and target
    X = data
    y = target

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Build Theil-Sen Regressor with scaling
    model = make_pipeline(
        StandardScaler(),
        TheilSenRegressor(random_state=42)
    )

    # Fit model
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)

    # Evaluate
    mape = mean_absolute_percentage_error(y_test, y_pred)
    print("Theil-Sen -> MAPE on test data : {:.4f}".format(mape))


In [ ]:
def TSR(data, target):
    """Theil-Sen Regression with MAPE evaluation for multi-output prediction."""

    # Split data into features and target
    X = data
    y = target

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )

    # Build Multi-Output Theil-Sen Regressor with scaling
    model = make_pipeline(
        StandardScaler(),
        MultiOutputRegressor(TheilSenRegressor(random_state=42))
    )

    # Fit model
    model.fit(X_train, y_train)

    # Predict
    y_pred = model.predict(X_test)

    # Evaluate - calculate MAPE for each target column
    mape_scores = []
    for i in range(y_test.shape[1]):
        mape = mean_absolute_percentage_error(y_test.iloc[:, i], y_pred[:, i])
        mape_scores.append(mape)
        print(f"Theil-Sen -> MAPE for BlendProperty{i+1}: {mape:.4f}")
    
    # Calculate average MAPE across all targets
    avg_mape = np.mean(mape_scores)
    print(f"Theil-Sen -> Average MAPE across all targets: {avg_mape:.4f}")
    
    return model

def predict_test_data(model, test_data):
    """Make predictions on test data and return results."""
    predictions = model.predict(test_data)
    return predictions

# Example usage:
# Load your data
train = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/train.csv")
test = pd.read_csv("/Users/MacbookPro/LocalStorage/Developer/ShellAi/dataset/test.csv")

# Prepare features and targets
X = train.iloc[:, :55]  # First 55 columns (blend composition + component properties)
y = train.iloc[:, 55:]  # Last 10 columns (BlendProperty1-10)

# Train the model
model = TSR(X, y)



Theil-Sen -> MAPE for BlendProperty1: 16.8220
Theil-Sen -> MAPE for BlendProperty2: 1.4078
Theil-Sen -> MAPE for BlendProperty3: 1.8033
Theil-Sen -> MAPE for BlendProperty4: 1.3816
Theil-Sen -> MAPE for BlendProperty5: 3.5232
Theil-Sen -> MAPE for BlendProperty6: 2.5212
Theil-Sen -> MAPE for BlendProperty7: 1.4006
Theil-Sen -> MAPE for BlendProperty8: 1.7987
Theil-Sen -> MAPE for BlendProperty9: 2.1516
Theil-Sen -> MAPE for BlendProperty10: 0.9838
Theil-Sen -> Average MAPE across all targets: 3.3794


ValueError: The feature names should match those that were passed during fit.
Feature names unseen at fit time:
- ID
Feature names seen at fit time, yet now missing:
- Component5_Property10


In [10]:
# Make predictions on test data

# test_ids = test["ID"]
# test = test.drop("ID", axis=1)
test_predictions = predict_test_data(model, test)

# Create submission DataFrame
submission = pd.DataFrame(test_predictions, 
                         columns=[f'BlendProperty{i}' for i in range(1, 11)])
# submission.insert(0, 'ID', test['ID'])  # Add ID column if present
submission.to_csv('submission.csv', index=True
                  )